# Flight Delay Prediction with Distributed Deep Learning
In this notebook we'll use a flight dataset from [here](https://kt.ijs.si/elena_ikonomovska/data.html). The data containsflight arrival and departure details for all the commercial flights within the USA, from October 1987 to April 2008. Its has about 117 million records and is about 6Gb in size

Rather that a traditional MLP architecture, we'll instead train a TabNet model. TabNet is a deep neural network model for Tabular Data proposed by Google [link](https://arxiv.org/abs/1908.07442).It utilizes transformer blocks and attention mechanism that have had significant success in the NLP and Computer Vision Domains. We'll also utilize [Ray](https://docs.ray.io/en/latest/index.html) for distributed data loading and preprocessing

![tabnet](image/tabnet.png)

## Advantages of Tabnet

- Train multi-objective models (e.g. multi-label, multi-regressor)
- Uses attention mechnism for feature selection. Which can also be used to provide local explanations at inference time
- It uses attention for selecting out the set of features to focus on for a given particular data point and we can even visualize that to see which parts get attention for a particular decision . We can also play with the number of features we want the Tabnet to focus to.
- Like other DL models, can easily utilize custom loss functions
- Can be pre-trained and finetuned similar to language and vision transformer models
- Can be trained on large number of features as it performs automatic feature selection which reduces a the need to do this during preprocessing
- Can be trained on large datasets without having to load the entire data into memory 
- Potentially requires less feature engineering

## Download Data
Download data from a public S3 bucket

In [ ]:
!wget http://ee-assets-prod-us-east-1.s3.amazonaws.com/modules/05fa7598d4d44836a42fde79b26568b2/v2/airline_14col.data.bz2 -P data/

## Prepare Data
- divide the single CSV file into multiple parquet files
- split into train and test datasets
- capture categorical feature data for future encoding purposes

In [ ]:
import pandas as pd
import numpy as np
import json
from pathlib import Path
from tqdm.notebook import tqdm

In [ ]:
cwd = Path.cwd()
data_path = cwd / "data"
train_data_path = data_path / "train"
test_data_path = data_path / "test"

train_data_path.mkdir(exist_ok=True)
test_data_path.mkdir(exist_ok=True)

In [ ]:
cols = [
        "Year", "Month", "DayofMonth", "DayofWeek", "CRSDepTime",
        "CRSArrTime", "UniqueCarrier", "FlightNum", "ActualElapsedTime",
        "Origin", "Dest", "Distance", "Diverted", "ArrDelay"
        ]

In [ ]:
df_sample = pd.read_csv(data_path / "airline_14col.data.bz2", nrows=100, names=cols)
df_sample.head()

In [ ]:
cat_cols = ["Month", "DayofMonth", "DayofWeek", "UniqueCarrier", "Origin", "Dest", "Diverted"]
cat_unique_values = {cat_col: set() for cat_col in cat_cols} # capture unique values for each categorical column

In [ ]:
chunks = pd.read_csv("data/airline_14col.data.bz2", chunksize=2_000_000, names=cols)

In [ ]:
# Converting the data into parquet chunks
for n,chunk in tqdm(enumerate(chunks), desc="Converting to parquet", total=58) :
    for col in cat_cols:
        cat_unique_values[col].update(set(chunk[col].unique()))
    if chunk.iloc[0]["Year"] < 2004:
        chunk.to_parquet(train_data_path / f"{n}.snappy.parquet", index=False)
    else:
        chunk.to_parquet(test_data_path / f"{n}.snappy.parquet", index=False)

In [ ]:
!rm data/airline_14col.data.bz2

In [ ]:
for col in ["Month", "DayofMonth", "DayofWeek", "Diverted"]:
    cat_unique_values[col] = set(map(int, cat_unique_values[col]))

cat_num_unique = {k: len(v) for k,v in cat_unique_values.items()}
cat_encoders = {k: dict(zip(cat_unique_values[k], range(len(cat_unique_values[k])))) for k in cat_unique_values}
cat_embed_size = {k: int(np.log1p(len(v)))+1 for k,v in cat_unique_values.items()}

In [ ]:
cat_col_meta_path = cwd / "col_meta"
cat_col_meta_path.mkdir(exist_ok=True)

(cat_col_meta_path / "encoders.json").open("w").write(json.dumps(cat_encoders))
(cat_col_meta_path / "embed_size.json").open("w").write(json.dumps(cat_embed_size))
(cat_col_meta_path / "num_unique.json").open("w").write(json.dumps(cat_num_unique))

## Train model with PyTorch and TensorFlow

In [ ]:
import boto3
import uuid
import shutil
import sagemaker
from sagemaker.pytorch import PyTorch
from sagemaker.tensorflow import TensorFlow

sess = sagemaker.session.Session()
role = sagemaker.get_execution_role()
bucket = sess.default_bucket()
key_prefix = f"{uuid.getnode()}/distributed_tabnet"

In [ ]:
# upload data to s3
s3_train_data_path = sess.upload_data("data/train/", bucket=bucket, key_prefix=f"{key_prefix}/train")
s3_test_data_path = sess.upload_data("data/test/", bucket=bucket, key_prefix=f"{key_prefix}/test")
s3_meta_data_path = sess.upload_data("col_meta/", bucket=bucket, key_prefix=f"{key_prefix}/meta")

In [ ]:
# optionally set subnets and security_groups
subnets=None
security_group_ids=None

In [ ]:
# Training with TensorFlow
tb_logging_path = f"s3://{bucket}/{key_prefix}/tb_logs/tf"
shutil.copyfile("src/requirements_tf.txt", "src/requirements.txt")
tf_estimator = TensorFlow(
    source_dir = "src",
    entry_point="train_airlines_tf.py",
    subnets=subnets,
    security_group_ids=security_group_ids,
    role=role,
    instance_count=1,
    hyperparameters={"s3_train_data":s3_train_data_path, 
                     "s3_test_data":s3_test_data_path, 
                     "s3_schema_file":f"{s3_train_data_path}/0.snappy.parquet",
                     "epochs":2, "batch_size": 50_000, "lr": 2e-2},
    instance_type="ml.g4dn.12xlarge", # try with ml.g5.12xlarge if limit exception raised
    framework_version="2.8",
    py_version="py39",
    checkpoint_s3_uri=tb_logging_path,
    keep_alive_period_in_seconds=1800
)

tf_estimator.fit({"meta": s3_meta_data_path}, wait=False) # change wait=True if you wnat to see the logs

### Analyze the model with TensorBoard
**Note: You have to wait a few minutes for the job to launch before seeing any logs**

We can use [TensorBoard](https://www.tensorflow.org/tensorboard), a visualization toolkit for analyzing deep learning models. Instructions for using TensorBoard with SageMaker Studio can be found [here](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-tensorboard.html). Instructions for accessing TensorBoard in SageMaker Studio are provided below:
1. Open a new terminal in SageMaker Studio by navigating to <em>File->New->Terminal <br> ![](./image/open_terminal.JPG)
2. Run the notebook cell below to generate a terminal command
3. Copy the command and paste it into the terminal and hit Enter
4. Return to the notebook an click the link provided in the bellow cell

In [ ]:
from IPython.display import HTML
import os
cur_dir = os.getcwd().replace(os.environ["HOME"],"")
HTML(f'''1. Paste the following command into the Studio Terminal <code style="background-color:grey;">tensorboard --logdir {tb_logging_path}</code>
<br>
(You may need to run <code style="background-color:grey;">pip install tensorboard</code> and <code style="background-color:grey;">pip install tensorflow_io</code> if TesorBoard is not already installed)
<br>
2. Click <a href='/jupyter/default/proxy/6006/'>here</a> to open TensorBoard''')

In [ ]:
# Training with PyTorch
tb_logging_path = f"s3://{bucket}/{key_prefix}/tb_logs/pt"
shutil.copyfile("src/requirements_pt.txt", "src/requirements.txt")
pt_estimator = PyTorch(
    source_dir = "src",
    entry_point="train_airlines_pt.py",
    subnets=subnets,
    security_group_ids=security_group_ids,
    role=role,
    instance_count=1, 
    instance_type="ml.g4dn.12xlarge", # try with ml.g5.12xlarge if limit exception raised
    framework_version="1.10",
    py_version="py38",
    hyperparameters={"s3_train_data":s3_train_data_path, 
                     "s3_test_data":s3_test_data_path, 
                     "s3_schema_file":f"{s3_train_data_path}/0.snappy.parquet",
                     "tb_logging_path":tb_logging_path,
                     "epochs":3, "batch_size": 50_000, "lr": 2e-2},
    keep_alive_period_in_seconds=1800
)

pt_estimator.fit({"meta": s3_meta_data_path}, wait=False) # change wait=True if you wnat to see the logs

In [ ]:
cur_dir = os.getcwd().replace(os.environ["HOME"],"")
HTML(f'''1. Paste the following command into the Studio Terminal <code style="background-color:grey;">tensorboard --logdir {tb_logging_path}</code>
<br>
(You may need to run <code style="background-color:grey;">pip install tensorboard</code> and <code style="background-color:grey;">pip install tensorflow_io</code> if TesorBoard is not already installed)
<br>
2. Click <a href='/jupyter/default/proxy/6006/'>here</a> to open TensorBoard''')

## Cleanup

In [ ]:
!rm -rf data col_meta/

In [ ]:
!aws s3 rm --recursive s3://{bucket}/{key_prefix}